Berikut adalah dokumentasi dalam format Markdown untuk proyek klasifikasi sampah menggunakan ResNet, diikuti oleh implementasi kode.

---

# Klasifikasi Sampah Menggunakan ResNet dengan TensorFlow

## Pendahuluan
Klasifikasi sampah menggunakan deep learning adalah metode yang dapat membantu otomatisasi dalam pemilahan dan pengelolaan limbah secara efisien. Model berbasis arsitektur Residual Network (ResNet) mampu mengenali jenis sampah dengan menggunakan dataset gambar yang dilabeli, yang berpotensi memperbaiki proses daur ulang otomatis. ResNet sering digunakan dalam pengenalan gambar karena kemampuannya mengatasi masalah vanishing gradient melalui blok residual yang mempertahankan informasi penting dari lapisan sebelumnya [(He et al., 2016)](https://consensus.app/papers/resnet-deep-learning-he/9c586abb395db4c2a24d1a4e18388f23/).

## Lingkungan dan Pengaturan Awal
- **Library yang Digunakan**: TensorFlow digunakan sebagai framework utama untuk deep learning, bersama pustaka `pandas`, `numpy`, dan `matplotlib` untuk manipulasi data dan visualisasi. `wandb` digunakan untuk pelacakan metrik model dan logging eksperimen, yang membantu dalam memantau performa model dan stabilitas selama pelatihan [(Biewald, 2020)](https://consensus.app/papers/weights-biases-open-source-biewald/e1289d2819c3a3d8cb70d44f6a5587b5/).
- **Dataset TrashNet**: Dataset gambar dari berbagai jenis sampah, seperti kertas, plastik, dan kaca, digunakan untuk melatih model. TrashNet menyediakan data yang beragam dan relevan untuk kebutuhan klasifikasi sampah otomatis [(Yang & Thung, 2017)](https://consensus.app/papers/classification-trash-using-deep-learning-yang-thung/e65627e5d9095e9232ff2f193fef10a7/).

## Arsitektur Model ResNet
ResNet menggunakan struktur blok residual untuk memungkinkan sinyal informasi melewati jaringan dengan minimal gangguan, mengatasi masalah vanishing gradient yang umum pada jaringan dalam [(He et al., 2016)](https://consensus.app/papers/resnet-deep-learning-he/9c586abb395db4c2a24d1a4e18388f23/).

- **Blok Residual**: Blok residual memungkinkan shortcut connections yang membawa input asli dari satu lapisan ke lapisan berikutnya, menjaga integritas informasi dari lapisan awal [(Szegedy et al., 2015)](https://consensus.app/papers/going-deeper-convolutions-szegedy/1a4e1e4b0f45310d711eaac8ae945a17/).
- **Arsitektur Model**: Model dimulai dengan lapisan konvolusi besar, diikuti dengan pooling dan beberapa blok residual. Global Average Pooling digunakan di akhir jaringan, yang memberikan stabilitas saat menangani data gambar dalam jumlah besar dan bervariasi.

## Pembagian Data
Untuk melatih model dengan data yang seimbang, dataset dibagi menjadi tiga set: pelatihan, validasi, dan pengujian. Pembagian manual memastikan data terdistribusi secara merata, meminimalkan risiko bias [(James et al., 2013)](https://consensus.app/papers/introduction-statistical-learning-james-et-al/09d17878513a84a3e95c748c8b6e5e93/).

## Preprocessing dan Augmentasi Data
- **Normalisasi dan Deteksi Blur**: Gambar normalisasi dan deteksi blur diterapkan pada setiap gambar agar kualitas gambar menjadi lebih baik sebelum dimasukkan ke model. Normalisasi dapat meningkatkan akurasi dan stabilitas model [(Krizhevsky et al., 2012)](https://consensus.app/papers/image-net-classification-deep-convolutional-krizhevsky-sutskever-hinton/2e5c8ac2f47d97cf9fc9b515f9aa2b24/).
- **Augmentasi Data**: Augmentasi seperti flip horizontal, rotasi, dan penyesuaian kecerahan digunakan untuk memperkaya data gambar. Teknik ini terbukti meningkatkan kemampuan generalisasi model pada data yang belum pernah dilihat sebelumnya [(Shorten & Khoshgoftaar, 2019)](https://consensus.app/papers/survey-image-data-augmentation-deep-shorten-khoshgoftaar/c2c52f2dd34891f128cc27ff18bc9e48/).

## Pelatihan Model
Model dilatih pada data pelatihan dan dievaluasi pada data validasi untuk menguji akurasi dan stabilitas. Hanya model yang mencapai akurasi dan stabilitas tertentu yang disimpan, untuk menghindari model overfitting atau underfitting [(Goodfellow et al., 2016)](https://consensus.app/papers/deep-learning-goodfellow-bengio-courville/90a54b117d70e0309978f7d0af87a4ef/).

## Evaluasi dan Prediksi Model
Evaluasi model menggunakan data pengujian untuk menghitung loss dan akurasi, memberikan gambaran yang lebih baik tentang performa model pada data baru. Akurasi tinggi menunjukkan kemampuan model dalam mengklasifikasikan sampah dengan baik [(Russakovsky et al., 2015)](https://consensus.app/papers/imagenet-large-scale-visual-recognition-challenge-russakovsky-deng/66e0a771282f27a07b05f98f315d2be3/).

---

In [5]:
import tensorflow as tf # type: ignore
import numpy as np
import random
from PIL import Image
from tensorflow.keras.preprocessing import image  # type: ignore
from tensorflow.keras.applications import ResNet50  # type: ignore
from tensorflow.keras import layers, models  # type: ignore
import wandb # type: ignore
import tensorflow as tf # type: ignore
import sys
import numpy as np
import random
import wandb
import os
from tensorflow.keras.optimizers import Adam # type: ignore
from tensorflow.keras.callbacks import ModelCheckpoint # type: ignore


# Memeriksa dan mengatur GPU
gpus = tf.config.list_physical_devices('GPU')
if not gpus:
    print("Tidak ada GPU yang terdeteksi. Program ini memerlukan GPU untuk dijalankan.")
    sys.exit(1)  # Menghentikan program dengan kode status 1

try:
    tf.config.experimental.set_memory_growth(gpus[0], True)
    print("Pengaturan memori GPU berhasil.")
except Exception as e:
    print(f"Kesalahan saat mengatur memori GPU: {e}")
    sys.exit(1)  # Menghentikan program jika terjadi kesalahan saat mengatur GPU


Pengaturan memori GPU berhasil.


In [ ]:
# Fungsi untuk memproses dan melakukan augmentasi pada gambar
import os
import random
import numpy as np
from tensorflow.keras.preprocessing import image # type: ignore
from tensorflow.keras.applications.resnet50 import ResNet50 # type: ignore
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense # type: ignore
from tensorflow.keras.models import Model # type: ignore
from tensorflow.keras.optimizers import Adam # type: ignore
from datasets import load_dataset # type: ignore
import wandb

def preprocess_and_augment_images(dataset, num_images=0, target_size=(224, 224)):
    train_images = []
    train_labels = []

    for i in range(min(num_images, len(dataset['train']))):
        try:
            img = dataset['train'][i]['image'].convert('RGB').resize(target_size)
            label = dataset['train'][i]['label']
            img_array = np.array(img) / 255.0
            
            # Deteksi blur dan augmentasi gambar
            if np.std(img_array) < 0.05:
                img_array *= 2  # Tingkatkan kontras
            
            augment_type = random.choice(['flip', 'rotate', 'brightness'])
            if augment_type == 'flip':
                img_array = np.fliplr(img_array)
            elif augment_type == 'rotate':
                img_array = np.rot90(img_array)
            elif augment_type == 'brightness':
                factor = random.uniform(0.5, 1.5)
                img_array = np.clip(img_array * factor, 0, 1)

            train_images.append(img_array)
            train_labels.append(label)

        except Exception as e:
            print(f"Error processing image {i + 1}: {e}")

    return np.array(train_images), np.array(train_labels)

# Fungsi untuk membuat model ResNet50
def create_resnet50(input_shape, num_classes):
    base_model = ResNet50(weights='imagenet', include_top=False, input_shape=input_shape)
    x = base_model .output
    x = GlobalAveragePooling2D()(x)
    outputs = Dense(num_classes, activation='softmax')(x)

    model = Model(inputs=base_model.input, outputs=outputs)

    for layer in base_model.layers:
        layer.trainable = False  # Membekukan lapisan dasar

    return model

# Fungsi untuk melatih dan menyimpan model dengan kondisi tertentu
def train_and_save_model(train_images, train_labels, val_images, val_labels, num_classes, epochs=10):
    # Membuat model menggunakan ResNet50
    model = create_resnet50(input_shape=(224, 224, 3), num_classes=num_classes)

    # Mengompilasi model dengan optimizer dan loss function
    model.compile(optimizer=Adam(), loss='sparse_categorical_crossentropy', metrics=['accuracy'])

    # Melatih model dan menyimpan history
    history = model.fit(train_images, train_labels, validation_data=(val_images, val_labels), epochs=epochs, batch_size=64)

    if history.history['val_accuracy'][-1] >= 0.80 and history.history['accuracy'][-1] >= 0.80 and abs(history.history['val_accuracy'][-1]-history.history['accuracy'][-1]) >= 0.01 :
        # Log ke wandb
        wandb.log({
            "train_accuracy": history.history['accuracy'][-1],
            "val_accuracy": history.history['val_accuracy'][-1],
            "train_loss": history.history['loss'][-1],
            "val_loss": history.history['val_loss'][-1]
        })
        model.save('simpan_resnet50_model_best.h5')
    # Memeriksa akurasi pada epoch terakhir
   
    model.save('simpan_resnet50_model.h5')  # Simpan model jika akurasi validasi <= 80%
    
    return model

# Fungsi untuk mengevaluasi model pada data uji
def evaluate_model(model, test_images, test_labels):
    loss, accuracy = model.evaluate(test_images, test_labels)
    print(f"Test Loss: {loss:.4f}, Test Accuracy: {accuracy:.4f}")

# Fungsi untuk prediksi gambar
def predict_image(model, img_path):
    img = image.load_img(img_path, target_size=(224, 224))
    img_array = np.array(img) / 255.0
    pred = model.predict(np.expand_dims(img_array, axis=0))
    predicted_class = np.argmax(pred, axis=-1)
    return predicted_class

# Fungsi utama untuk menjalankan pelatihan dan evaluasi
def main():
    # Inisialisasi wandb dan login menggunakan API Key
    wandb.login(key='967fafd5558eeeff3ce5681cf55c71633438428d')  # Ganti dengan API Key kamu
   
    wandb.init(project="trash-classification")
    # Load dataset TrashNet dari Hugging Face
    dataset = load_dataset("garythung/trashnet")

    # Preprocessing dan augmentasi data
    train_images, train_labels = preprocess_and_augment_images(dataset, num_images=1000)
    val_images, val_labels = preprocess_and_augment_images(dataset, num_images=200)
    
    # Melatih dan menyimpan model
    modelVal = train_and_save_model(train_images, train_labels, val_images, val_labels, num_classes=6, epochs=400)
   
    # Evaluasi model pada data uji
    test_images, test_labels = preprocess_and_augment_images(dataset, num_images=100)
    evaluate_model(modelVal, test_images, test_labels)

    # Prediksi gambar
    predicted_class = predict_image(modelVal, "/workspaces/aitask/notebooks/image/sampah.jpg")
    print(predicted_class)

    # Logout wandb setelah eksperimen selesai
    wandb.finish()

# Menjalankan fungsi utama
if __name__ == "__main__":
    main()

wandb: WARNING Calling wandb.login() after wandb.init() has no effect.


wandb: WARNING Source type is set to 'repo' but some required information is missing from the environment. A job will not be created from this run. See https://docs.wandb.ai/guides/launch/create-job


Epoch 1/400
16/16 [==============================] - 7s 270ms/step - loss: 1.2729 - accuracy: 0.4080 - val_loss: 0.8405 - val_accuracy: 0.0050
Epoch 2/400
16/16 [==============================] - 3s 176ms/step - loss: 0.9594 - accuracy: 0.4760 - val_loss: 1.0663 - val_accuracy: 0.0000e+00
Epoch 3/400
16/16 [==============================] - 3s 175ms/step - loss: 0.9520 - accuracy: 0.4990 - val_loss: 1.0776 - val_accuracy: 0.0000e+00
Epoch 4/400
16/16 [==============================] - 3s 176ms/step - loss: 0.9364 - accuracy: 0.5120 - val_loss: 1.0217 - val_accuracy: 0.0000e+00
Epoch 5/400
16/16 [==============================] - 3s 175ms/step - loss: 0.9262 - accuracy: 0.5410 - val_loss: 0.8099 - val_accuracy: 0.6200
Epoch 6/400
16/16 [==============================] - 3s 175ms/step - loss: 0.9138 - accuracy: 0.5300 - val_loss: 0.8084 - val_accuracy: 0.5650
Epoch 7/400
16/16 [==============================] - 3s 176ms/step - loss: 0.9070 - accuracy: 0.5590 - val_loss: 1.0413 - val_accu

/usr/local/lib/python3.11/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


4/4 [==============================] - 0s 59ms/step - loss: 0.7269 - accuracy: 0.6500
Test Loss: 0.7269, Test Accuracy: 0.6500


FileNotFoundError: [Errno 2] No such file or directory: '/workspaces/aitask/notebooks/image/sampah.jpg'

Error in callback <bound method _WandbInit._pause_backend of <wandb.sdk.wandb_init._WandbInit object at 0x7f36d8399010>> (for post_run_cell), with arguments args (<ExecutionResult object at 7f37546b0910, execution_count=6 error_before_exec=None error_in_exec=[Errno 2] No such file or directory: '/workspaces/aitask/notebooks/image/sampah.jpg' info=<ExecutionInfo object at 7f36d87dc650, raw_cell="# Fungsi untuk memproses dan melakukan augmentasi .." store_history=True silent=False shell_futures=True cell_id=vscode-notebook-cell://codespaces%2Bimproved-happiness-xvwwwq9gj4rhpvq/workspaces/aitask/notebooks/Trash_Classification.ipynb#W2sdnNjb2RlLXJlbW90ZQ%3D%3D> result=None>,),kwargs {}:


TypeError: _WandbInit._pause_backend() takes 1 positional argument but 2 were given

Jika Hadi